In [ ]:
Authors
- Shawn Fan
- Artur Dox
- Ang Li
- 

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import anndata
from metrics import *
from de import *
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

AttributeError: partially initialized module 'jax' has no attribute 'version' (most likely due to a circular import)

In [3]:
data_file = '../datasets/data/1gene-norman.h5ad'
adata = anndata.read_h5ad(data_file)
adata_control = adata[adata.obs['perturbation_name'] == 'control']
control_mean = np.array(adata_control.X.mean(axis=0))[0]

# Select subset of perturbations for testing purposes
adata_subset = adata[adata.obs['perturbation_name'].isin(['FOXA1', 'FOXA3'])]

# Simulate split (ground truth cells, predicted cells)
idxs_gt, idxs_pred = train_test_split(np.arange(len(adata_subset)), test_size=0.5)
adata_gt = adata_subset[idxs_gt]
adata_pred = adata_subset[idxs_pred]

In [4]:
# Run DE analysis
de_genes_gt = get_de_genes(adata_control, adata_gt,
                           method = "wilcoxon",
                           top_k = 100,
                           groupby_key = 'perturbation_name')
de_genes_pred = get_de_genes(adata_control, adata_pred,
                             method = "wilcoxon",
                             top_k = 100,
                             groupby_key = 'perturbation_name')

/home/vinas/scEvalsJam/metrics/de.py:41: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  anndata_x.obs["group"] = "control"
/home/vinas/scEvalsJam/metrics/de.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  anndata_y.obs["group"] = "perturbation"
/home/vinas/scEvalsJam/metrics/de.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  anndata_y.obs["group"] = "perturbation"


In [23]:
unique_perturbations = adata_subset.obs['perturbation_name'].unique()
results_df = pd.DataFrame()
results_list = []
for pert in tqdm(unique_perturbations):    
    # Select data from relevant perturbation
    adata_gt_pert = adata_gt[adata_gt.obs['perturbation_name'] == pert]
    adata_pred_pert = adata_pred[adata_pred.obs['perturbation_name'] == pert]
    de_up_mask = adata_gt.var.index.isin(de_genes_gt[pert]['de_up'])
    de_dn_mask = adata_gt.var.index.isin(de_genes_gt[pert]['de_dn'])
    
    # Compute metrics
    pred = adata_pred_pert.X.toarray()
    gt = adata_gt_pert.X.toarray()
    ec_pred = np.array(adata_pred_pert.X-control_mean)
    ec_gt = np.array(adata_gt_pert.X-control_mean)
    pert_scores = []
    for metric_name, metric_fn in metric_dict.items():
        score = metric_fn(pred, gt)
        score_topk_up_deg = metric_fn(pred[:, de_up_mask], gt[:, de_up_mask])
        score_topk_dn_deg = metric_fn(pred[:, de_dn_mask], gt[:, de_dn_mask])
        
        score_ec = metric_fn(ec_pred, ec_gt)
        score_ec_topk_up_deg = metric_fn(ec_pred[:, de_up_mask], ec_gt[:, de_up_mask])
        score_ec_topk_dn_deg = metric_fn(ec_pred[:, de_dn_mask], ec_gt[:, de_dn_mask])
        pert_scores.extend([score, score_topk_up_deg, score_topk_dn_deg, score_ec, score_ec_topk_up_deg, score_ec_topk_dn_deg])
        
        if metric_name in ['mse', 'mae', 'euclidean_distance']:
            assert score_ec == score and score_ec_topk_up_deg == score_topk_up_deg and score_ec_topk_dn_deg == score_topk_dn_deg

    # Jaccard index between gene sets
    jaccard_up = jaccard_similarity(de_genes_gt[pert]['de_up'], de_genes_pred[pert]['de_up'])
    jaccard_dn = jaccard_similarity(de_genes_gt[pert]['de_dn'], de_genes_pred[pert]['de_dn'])
    pert_scores.extend([jaccard_up, jaccard_dn])
    
    # Append perturbation scores
    results_list.append(pert_scores)

column_names = []
for metric_name, _ in metric_dict.items():
    column_names.extend([f'{metric_name}',
                        f'{metric_name}_up_deg',
                        f'{metric_name}_dn_deg',
                        f'{metric_name}_ec',
                        f'{metric_name}_ec_up_deg',
                        f'{metric_name}_ec_dn_deg'])
column_names.extend(['jaccard_up', 'jaccard_dn'])

# Store results
results_df = pd.DataFrame(results_list,
                          index=unique_perturbations, 
                          columns=column_names)  # .loc[len(results_df)] = pd.DataFrame(scores)


  0%|                                                                                                        | 0/2 [00:00<?, ?it/s]/home/vinas/miniconda3/envs/scpert/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/vinas/scEvalsJam/metrics/metrics.py:37: RuntimeWarning: invalid value encountered in sqrt
  BC = np.sum(np.sqrt(X_norm * Y_norm))
/home/vinas/scEvalsJam/metrics/metrics.py:54: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - np.mean(intersection / union)
/home/vinas/miniconda3/envs/scpert/lib/python3.12/

mse
mae
ks_test_distance
edistance
cosine_distance
pearson_distance
euclidean_distance
classifier_proba
kendalltau_distance
spearman_distance
wasserstein
sym_kldiv
bhattacharyya_distance
jaccard_index
F1_score
mutual_information


In [24]:
results_df

,mse,mse_up_deg,mse_dn_deg,mse_ec,mse_ec_up_deg,mse_ec_dn_deg,mae,mae_up_deg,mae_dn_deg,mae_ec,...,F1_score_ec_up_deg,F1_score_ec_dn_deg,mutual_information,mutual_information_up_deg,mutual_information_dn_deg,mutual_information_ec,mutual_information_ec_up_deg,mutual_information_ec_dn_deg,jaccard_up,jaccard_dn
FOXA1,0.000326,0.001018,0.001265,0.000326,0.001018,0.001265,0.010736,0.026327,0.028431,0.010736,...,0.0,0.0,4.880406,4.605170,4.60517,8.655222,4.60517,4.60517,0.379310,0.503759
FOXA3,0.000589,0.002691,0.002281,0.000589,0.002691,0.002281,0.014343,0.039317,0.038840,0.014343,...,0.0,0.0,4.326426,4.591307,4.60517,8.561992,4.60517,4.60517,0.408451,0.459854


In [ ]:
redundant_column_mask = results_df.columns.str.contains('ec') & results_df.columns.str.contains('mse|mae|euclidean_distance|bhattacharyya_distance')

results_df = results_df.loc[:, ~redundant_column_mask]

In [ ]:
results_df